In [61]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
import pandas as pd

In [62]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [63]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("make_vector_DB")

LangSmith 추적을 시작합니다.
[프로젝트명]
make_vector_DB


In [64]:
# 임베딩할 데이터셋 정하기
DATA_PATH="../datas/merge_data/"
DATA_NAME="음식테마거리+7BEACH_가게"
DATA_EXTENTION=".xlsx"

In [65]:


# Excel 파일 읽기
df = pd.read_excel(DATA_PATH + DATA_NAME + DATA_EXTENTION)
df = df.head(100)


In [66]:
model = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [67]:
# 컬럼 데이터
column_data = ', '.join(df.columns)

# 예시 행 데이터
example_row_data = ', '.join(map(str, df.iloc[0].tolist()))


In [68]:
question = \
"""
주어진 Column data을 각 컬럼을 key값, 구어체로 번역된 컬럼을 value값으로 만들어줘
예시 데이터를 줄테니 알맞은 컬럼 이름을 생성해줘.
# 예시
'원래 컬럼': '구어체로 번역된 컬럼'

"""

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 영어로 된 컬럼이름을 한국어로 바꾸어주는 어시스턴트 입니다."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}\n\n#Column data: {column_data}\n\n#Example Column data: {example_column_data}"),
    ]
)

# 지시사항을 프롬프트에 주입합니다.
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 프롬프트, 모델, 파서를 연결하는 체인 생성
chain = prompt | model | parser

# 체인을 호출하여 쿼리 실행
column_vocab = chain.invoke({"question": question, "column_data": column_data, "example_column_data": example_row_data})

# 출력을 확인합니다.
print(column_vocab)


{'RSTR_ID': '식당 ID', 'RSTR_NM': '식당 이름', 'RSTR_RDNMADR': '식당 도로명 주소', 'RSTR_LNNO_ADRES': '식당 지번 주소', 'RSTR_LA': '식당 위도', 'RSTR_LO': '식당 경도', 'RSTR_TELNO': '식당 전화번호', 'BSNS_STATM_BZCND_NM': '영업 상태 및 업종명', 'BSNS_LCNC_NM': '영업 허가명', 'RSTR_INTRCN_CONT': '식당 소개 내용', 'AREA_NM': '지역 이름', 'PRDL_SEAT_CNT': '상품 좌석 수', 'SEAT_CNT': '좌석 수', 'PRKG_POS_YN': '주차 가능 여부', 'WIFI_OFR_YN': '와이파이 제공 여부', 'DCRN_YN': '할인 여부', 'PET_ENTRN_POSBL_YN': '반려동물 입장 가능 여부', 'FGGG_MENU_OFR_YN': '외국어 메뉴 제공 여부', 'TLROM_INFO_CN': '화장실 정보', 'RESTDY_INFO_CN': '휴무일 정보', 'BSNS_TM_CN': '영업 시간', 'HMDLV_SALE_YN': '포장 판매 여부', 'DSBR_CVNTL_YN': '배달 편의 여부', 'DELV_SRVIC_YN': '배달 서비스 여부', 'RSRV_MTHD_NM': '예약 방법', 'ONLINE_RSRV_INFO_CN': '온라인 예약 정보', 'HMPG_URL': '홈페이지 URL', 'CRCMF_LDMARK_NM': '주변 랜드마크 이름', 'CRCMF_LDMARK_LA': '주변 랜드마크 위도', 'CRCMF_LDMARK_LO': '주변 랜드마크 경도', 'CRCMF_LDMARK_DIST': '주변 랜드마크 거리', 'KIOSK_YN': '키오스크 여부', 'MB_PMAMT_YN': '멤버십 결제 여부', 'SMORDER_YN': '스마트 주문 여부', 'REPRSNT_MENU_NM': '대표 메뉴 이름', 'AWARD_INFO_DSCRN': '수상 정

In [69]:
# 문장 데이터에서 제외될 컬럼
exclude_columns = ['combine_column']

# Y, N같은 데이터는 문자열로 변환
replacement_map = {
    "Y": "가능합니다.",
    "N": "불가능합니다.",
}

# 데이터 프레임 행을을 문장으로 변형해서 컬럼 추가.
df["combine_column"] = df.apply(
    lambda row: ' '.join(
        [f"{column_vocab[col]}은 {replacement_map.get(row[col], '알 수 없습니다.' if pd.isna(row[col]) else str(row[col]) + '입니다.')}" for col in df.columns if col not in exclude_columns]
    ),
    axis=1
)

In [70]:
df["combine_column"][0]

'식당 ID은 1085입니다. 식당 이름은 커피탱크입니다. 식당 도로명 주소은 부산광역시 사하구 장림번영로 25입니다. 식당 지번 주소은 부산광역시 사하구 장림동 328-9입니다. 식당 위도은 35.0809885입니다. 식당 경도은 128.9682126입니다. 식당 전화번호은 051-263-4629입니다. 영업 상태 및 업종명은 다방입니다. 영업 허가명은 휴게음식점입니다. 식당 소개 내용은 부산광역시 사하구에서 어디를 갈지 고민이라면! "커피탱크"에 가보시는 건 어떨까요?입니다. 지역 이름은 부산광역시 사하구입니다. 상품 좌석 수은 알 수 없습니다. 좌석 수은 알 수 없습니다. 주차 가능 여부은 불가능합니다. 와이파이 제공 여부은 불가능합니다. 할인 여부은 불가능합니다. 반려동물 입장 가능 여부은 불가능합니다. 외국어 메뉴 제공 여부은 불가능합니다. 화장실 정보은 불가능합니다. 휴무일 정보은 알 수 없습니다. 영업 시간은 알 수 없습니다. 포장 판매 여부은 불가능합니다. 배달 편의 여부은 불가능합니다. 배달 서비스 여부은 불가능합니다. 예약 방법은 알 수 없습니다. 온라인 예약 정보은 알 수 없습니다. 홈페이지 URL은 알 수 없습니다. 주변 랜드마크 이름은 알 수 없습니다. 주변 랜드마크 위도은 알 수 없습니다. 주변 랜드마크 경도은 알 수 없습니다. 주변 랜드마크 거리은 알 수 없습니다. 키오스크 여부은 불가능합니다. 멤버십 결제 여부은 불가능합니다. 스마트 주문 여부은 불가능합니다. 대표 메뉴 이름은 알 수 없습니다. 수상 정보 설명은 알 수 없습니다. RTI 지수은 0.0입니다. 온라인 전환 진행 여부은 불가능합니다. 수용 상태 지수은 0.17입니다. 평점 지수은 0.0입니다. 트립어드바이저 등급은 0.0입니다. 씨트립 등급은 알 수 없습니다. 네이버 등급은 알 수 없습니다.'

In [71]:
from langchain_community.document_loaders import DataFrameLoader

# 데이터 프레임 로더 설정, 페이지 내용 컬럼 지정
loader = DataFrameLoader(df, page_content_column="combine_column")

# 문서 로드
docs = loader.load()


In [72]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [73]:
# DB 생성
db = FAISS.from_documents(documents=docs, embedding=embeddings)

In [ ]:
# 로컬 Disk 에 저장
db.save_local(folder_path="faiss_db", index_name=DATA_NAME)